In [164]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [165]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()


#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0
T7_DNA.lc =0

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [166]:

def run_model(model,t,parameters_list,molecules_0,dilute_list,replenish_list,result_all):  
    start_cycle,end_cycle = np.array(t)*4
    for n in range (start_cycle,end_cycle):
        #define time
        t_start= n*15
        t_end = (n+1)*15
        t= np.linspace(t_start,t_end,2)

        #solve equation and save result
        result = scipy.integrate.odeint(model, molecules_0, t, args=parameters_list)
        result_all = np.append(result_all,result[0])
        
        #update parameter
        molecules_0 = result.transpose()[:,-1]
        
        #dilution 
        ###diute out
        dilute_species((dilute_list),molecules_0)
        
        ###replenish 
        replenish_species((replenish_list),molecules_0)
    return result_all,molecules_0

In [167]:
def Repressor_model(molecules,t,alpha,beta,K,Kr,KTL,KTX,lamb1,lamb2):
    T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = molecules
    TX = 1
    TL = 1
    return np.array(
        [
            0,
            alpha*TX*T7_DNA*T7*R/(R+K),
            beta*TL*T7_RNA*(R/(R+K)),
            0,
            alpha*TX*GFP_DNA*T7*(R/(R+K)),
            beta*TL*GFP_RNA*(R/(R+K)),
            -alpha*TX*T7_DNA*T7*(R/(R+K))-alpha*TX*GFP_DNA*T7*(R/(R+K))-beta*TL*T7_RNA*(R/(R+K))-beta*TL*GFP_RNA*(R/(R+K)),
        ]
    )
    

In [169]:
#Positive control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [T7,R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,7))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).pc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(R.pc_result)
print(R.pc_result)


[100.         100.          74.77914516  41.23343112  20.00000003
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20. 

In [183]:
#Negative control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,7))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).nc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.nc_result)

In [189]:
#experiment 
T7_DNA.ic = 0.001
T7_DNA.lc = 0.001
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list

for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition

DR = 0.2

#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,GFP_DNA,T7_RNA,T7,GFP,GFP_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(121,7))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.result)
result3 = GFP.result

In [190]:

t = np.linspace(0, 15*(len(GFP.pc_result)-1), len(GFP.pc_result))
p = bokeh.plotting.figure(
    plot_width=600,
    plot_height=400,
    x_axis_label="t",
    y_axis_type="linear",
)

colors = bokeh.palettes.d3["Category10"][3]

# Populate glyphs
p.line(   t/60, GFP.pc_result, line_width=2, color=colors[2])
p.line(   t/60, GFP.nc_result, line_width=2, color=colors[1])
p.line(   t/60, result1, line_width=2, color=colors[0])
p.line(   t/60, result2, line_width=2, color=colors[0])
p.line(   t/60, result3, line_width=2, color=colors[0])
vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)
show(p)


In [184]:
result1

array([ 0.        ,  0.        , 11.67144619, 11.84204236, 11.95291563,
       12.02514609, 12.07227408, 12.10305395, 12.12316958, 12.13632127,
       12.14492226, 12.15054816, 12.15422848, 12.15663624, 12.15821153,
       12.15924222, 12.15991658, 12.16035782, 12.16064653, 12.20511473,
       12.27041031, 12.34234017, 12.41280557, 12.4775976 , 12.53489077,
       12.58425315, 12.62601702, 12.66089123, 12.68973079, 12.71340601,
       12.73273338, 12.74844354, 12.76117078, 12.77145452, 12.77974681,
       12.78642246, 12.79178977, 12.79610076, 12.79956053, 12.80233537,
       12.80455975, 12.80634213, 12.80776987, 12.80891325, 12.80982871,
       12.81056155, 12.81114814, 12.81161761, 12.81199331, 12.81229396,
       12.81253452, 12.81272701, 12.81288102, 12.81300424, 12.81310282,
       12.8131817 , 12.8132448 , 12.81329529, 12.81333568, 12.81336799,
       12.81339384, 12.81341452, 12.81343106, 12.8134443 , 12.81345489,
       12.81346336, 12.81347014, 13.22263322, 13.91908866, 14.81